# Ternary quantization demo
This notebook shows how a small dense layer (4×32) is quantized into balanced ternary limbs using `t81lib`.
The `pack_dense_matrix` helper clamps floats to [-1,1], writes the quantized trits into 16-byte limbs, and returns a buffer that can be passed directly to `t81lib.gemm_ternary`.


In [ ]:
import numpy as np
import t81lib

np.random.seed(0)
weights = np.random.uniform(-1.5, 1.5, (4, 32)).astype(np.float32)
packed = t81lib.pack_dense_matrix(weights, threshold=0.4)
print('Packed limbs shape:', packed.shape)

trits = t81lib.unpack_packed_limbs(packed, rows=4, cols=32)
print('Unique trits and counts:', np.unique(trits, return_counts=True))

dequantized = t81lib.dequantize_trits(trits)
print('Max reconstruction error:', np.max(np.abs(weights - dequantized)))

direct = t81lib.quantize_to_trits(weights, threshold=0.4)
print('Matches direct quantize?', np.array_equal(direct, trits))

print('GEMM ready: pass the packed buffer into t81lib.gemm_ternary with K=32 and an appropriately-shaped bias.')


In [ ]:
Limb = t81lib.Limb
BigInt = t81lib.BigInt
ctx = t81lib.BigIntMontgomeryContext(BigInt(17))
print('Montgomery exponentiation:', ctx.mod_pow(BigInt(5), BigInt(3)))
print('Limb consensus of 1 and -1:', Limb(1).consensus(Limb(-1)))
